![](images/logos/rosdevday.jpg)

# How to cross-compile ROS2 distro by taken VxWorks RTOS as an example

Even Open Robotics provides pre-built ROS 2 packages for multiple platforms, very often target software and hardware differ from the default one and a cross-compilation becomes a mandatory step:
* A different from Linux operating system e.g. VxWorks, QNX, eSol, etc. is deployed on the target hardware
* Target hardware (e.g. ARM aarch64) is different from the development host (e.g. Intel x86_64).
* Tuning target software for the footprint and performance (e.g. setting -mcpu=cortex-a72 -mfpu=neon-fp-armv8 when building for Raspberry Pi4).
* Separating ROS2 host tools (e.g. RViz) from the ROS2 target binaries.

This session will explain in detail of why corss-compilation is needed, and how to cross-compile ROS2 distro by taken VxWorks RTOS as an example. Step by step instructions will be given of how to setup a cross-compile development environment, to build and to deploy ROS2 binaries on the target. As a target QEMU ARM will be used on the Desktop PC.

[Indicate here the robot or hardware you are going to create the programs for]

[Substitute the image below by the image of your robot or hardware]

![Image obtained from IF World Design Guide](images/franka.jpg)

## WHO AM I

<img src="images/headshot.jpg" width="200">
<center><strong>Andrei Kholodnyi</strong> | <strong>Principal Technologist</strong> | <strong>Technology Office</strong> | <strong>Wind River Systems</strong></center>

### Focus

 *  ROS2 Mobile Robotics, Dependability
 *  Industrial, TSN, intelligent edge
 *  ROS2 Open Source Community (real-time WG co-chair)
 *  Products, Solutions; Partnerships & University Programs

## What is VxWorks RTOS

[Select the simulation you will use during the presentation. You can either use one of the provided by the Simulations menu above, or upload/clone your own. Your simulation must be placed on the simulation_ws/src directory. 
To upload, open the IDE on the Tools menu, and right click on the directory. then select upload
To clone, open a terminal on the Tools menu, navigate to the directory and do your git clone]

[Provide here instructions for the audience about how to launch the simulation. Delete this section if your presentation does not use a simulation]

For ex. instructions about how to launch the Franka Emika simulation (whose code you can find inside the *simulation_ws/src* directory of this rosject):

1. Go to the top menu and select Simulations
2. On the screen that appears, press the blue button that says *Choose launch file*
3. On the new box that appears, click on the drop down menu and select the launch file *simulation.launch*
4. After a few seconds, you will have a new window with a Gazebo simulation of the Panda robot

[substitute the image below by the expected window the audience will get after launching the simulation]

![](images/rosject_images/panda-in-gazebo.png)

## [Write your main subject title here]

[Write here the explanation of your subject. The shorter the better. You can use additional images or videos to make clearer your explanation]

## [Write the title of your practical demo]

[Describe here step by step how to do and replicate your experiment/program/code result. The steps must be clear and reproducible by the audience. The clearer they are, the easier it will be for you to follow along the notes during your presentation]

[Include paragraphs with code (any language) or command lines if necessary, like the following three]

[ex. of command line]

In [1]:
$ cd /home/user/simulation_ws/src

SyntaxError: invalid syntax (<ipython-input-1-85385d7399c1>, line 1)

[ex. of launch file]

In [ ]:
<?xml version="1.0"?>
<launch>
    <!-- maps inside the folder summit_xl_navigation/maps -->
    <arg name="map_file" default="map1/map1.yaml"/>
    <arg name="prefix" default="summit_xl_"/>
    <arg name="frame_id" default="$(arg prefix)map"/>

    <!-- Run the map server -->
    <node name="map_server" pkg="map_server" type="map_server" args="$(find summit_xl_localization)/maps/$(arg map_file)">
        <param name="frame_id" value="$(arg frame_id)"/>
    </node>

</launch>

[ex. of C++ code]

In [ ]:
if ( !inliers->indices.empty() )
{
    //Extract inliers
    pcl::ExtractIndices<PointT> extract;
    extract.setInputCloud(inputCloud);
    extract.setIndices(inliers);
    extract.setNegative(false);
    extract.filter(*cylinderCloud);
    return true;
}

[Include images of the expected results to help audience understand your point, like the following image]

![](images/rosject_images/sim7.png)

[If you need extra code already available to the audience (for example, in case you want to use an arm robot with MoveIt and you need the MoveIt configuration already done) you can upload the code and ROS packages to the catkin_ws/src directory using the IDE->Upload option]

[If you need to provide a dataset, follow instructions below about how to create/upload a dataset and provide it to audience]

## [Concluding remarks]


[Write here your conclusions]

# Examples of notebooks from previous speakers

* [ROSDevcon2019 TIAGo PCL table segmentation, by Luca Marchionni from Pal Robotics](notebook_examples/tiago/Course_TiagoInstructionManual_Unit7.ipynb)
* [ROSDevCon2018 ROS Navigation Stack, by Roman Navarro from Robotnik](notebook_examples/robotnik/robotnik_presentation.ipynb)
* [ROSDevCon2019 Unix Domain Socket Communication in ROS, by Fujita Tomoya from Sony](notebook_examples/sony/1-Overview.ipynb)

# Extra ROSject Documentation

[You can delete all the information in this section if you don't need it]

## How to add a dataset

Now let's see how to create a Dataset

go to the **Datasets** menu at the top of the window and select **create**

![](images/useful_images/dataset.png)

Fill all the data in order to create your dataset, the name of the dataset, the folder that will contain this dataset, some description , and select if you want that your dataset will be public or not

![](images/useful_images/dataset2.png)

Once you have already created, you will see that your new dataset is ready .

![](images/useful_images/dataset3.png)

If you go to the IDE, you will see your new dataset in the **datasets_ws** ready to store all your data

![](images/useful_images/dataset4.png)

Ok, now should be interesting to add some data to your new dataset, we will use **rosbag** to do it, so open a new Shell and navigate to the dataset first.

In [ ]:
$ cd datasets_ws/joints_states_values/

Then we will store all the information of a topic, in this case the topic called **/joint_states**, in order to do that run the following command in the Shell

In [ ]:
$ rosbag record /joint_states

This will record all the info of that topic, even if you move the robot again. 

you can move the robot again while the rosbag is recording

Once you want to stop the recording just press Ctrl +c in the shell to stop it

You can work with this information, for example let's see the info of the bag you have created, go to the Shell and run $ **rosbag info (the name of your bag)** you will have something similar to the next image.

![](images/rosject_images/rosbag.png)

But, let's see how it looks like in the IDE

![](images/rosject_images/rosbag2.png)

as you can see there is a new file, with the date of creation, and if you open it you will see some information, but most of the information is in other format that cannot read, but you can work with it.

#### You can upload already created datasets to this rosject

If you created **datasets** in other projects of yours, you can upload them, by using the IDE and uploading them to the directory datasets_ws/joint_states_values.

If you want to have the dataset in a different directory, create first the dataset directory using the *Datasets* menu on the top.